# QR DECOMPOSITION

# Content
QR Decomposition
- Orthogonal projection and least squares
- A Gram-Schmidt process
- Eigenvalues and eigenvectors

## Overview

This notebook describes the QR decomposition and how it relates to
- Orthogonal projection and least squares
- A Gram-Schmidt process
- Eigenvalues and eigenvectors

We’ll write some Python code to help consolidate our understandings.

## Matrix Factorization

The QR decomposition (also called the QR factorization) of a matrix is a decomposition of a matrix into the product of an orthogonal matrix and a triangular matrix. A QR decomposition of a real matrix $A$ takes the form
$$
A = QR
$$
where
- $Q$ is an orthogonal matrix (so that $Q^TQ=I$)
- $R$ is an upper triangular matrix

We’ll use a **Gram-Schmidt process** to compute a QR decomposition.

## Gram-Schmidt process

We’ll start with a **square** matrix $A$. If a square matrix $A$ is nonsingular, then a $QR$ factorization is unique. We’ll deal with a rectangular matrix $A$ later. Actually, the algorithm will work with a rectangular $A$ that is not square.

### Gram-Schmidt process for square $A$

Here we apply a Gram-Schmidt process to the **columns** of matrix $A$. In particular, let
$$
A = [a_1|a_2|\dots|a_n]
$$
Let $\Vert\dot\Vert$ denote the L2 norm. The Gram-Schmidt algorithm repeatedly combines the following two steps in a particular order
- **normalize** a vector to have unit norm
- **orthogonalize** the next vector

To begin, we set $u_1 = a_1$ and then **normalize**:
$$
u_1 = a_1,\text{ } e_1 = \frac{u_1}{\Vert u_1\Vert}
$$
We **orgonalize** first to compute $u_2$ and then **normalize** to create $e_2$:
$$
u_2 = a_2 - (a_2\cdot e_1)e_1,\text{ } e_2 = \frac{u_2}{\Vert u_2\Vert}
$$
We invite the reader to verify that $e_1\cdot e_2 = 0$, hence orthogonal. The Gram-Schmidt procedure continues iterating. Thus, for $k=2,\dots,n-1$ we construct
$$
u_2 = a_{k+1} - (a_{k+1}\cdot e_1)e_1 -\dots-(a_{k+1}\cdot e_k)e_k,\text{ } e_{k+1} = \frac{u_{k+1}}{\Vert u_{k+1}\Vert}
$$

Here $(a_j\cdot e_i)$ can be interpreted as the linear least squares **regression coefficient** of $a_j$ on $e_i$

- it is the inner product of $a_j$ on $e_i$ divided by the inner product of $e_i$ where $e_i\cdot e_i = 1$, as *normalization* has assured us.
- this regression coefficient has an interpretation as being a **covariance** divided by a **variance**

It can be verified that
$$
A = [a_1|a_2|\dots|a_n] = [e_1|e_2|\dots|e_n]
\begin{bmatrix}
a_1\cdot e_1 & a_2\cdot e_1 &\dots & a_n\cdot e_1\\
0 & a_2\cdot e_2 &\dots & a_n\cdot e_2\\
\vdots & \vdots & \ddots & \vdots\\
0 & 0 &\dots & a_n\cdot e_n\\
\end{bmatrix}
= QR
$$

where 
$$
Q = [e_1|e_2|\dots|e_n] \text{ and } R=\begin{bmatrix}
a_1\cdot e_1 & a_2\cdot e_1 &\dots & a_n\cdot e_1\\
0 & a_2\cdot e_2 &\dots & a_n\cdot e_2\\
\vdots & \vdots & \ddots & \vdots\\
0 & 0 &\dots & a_n\cdot e_n\\
\end{bmatrix}
$$

### $A$ not square

Now suppose that $A$ is an $n\times m$ matrix where $m > n$. Then a $QR$ decomposition is

$$
A = [a_1|a_2|\dots|a_m] = [e_1|e_2|\dots|e_n]
\begin{bmatrix}
a_1\cdot e_1 & a_2\cdot e_1 &\dots & a_n\cdot e_1 & \dots & a_m\cdot e_1\\
0 & a_2\cdot e_2 &\dots & a_n\cdot e_2 & \dots & a_m\cdot e_2\\
\vdots & \vdots & \ddots & \vdots & \dots & \ddots\\
0 & 0 &\dots & a_n\cdot e_n& \dots & a_m\cdot e_n\\
\end{bmatrix}
$$

## Some Code
Now let’s write some homemade Python code to implement a QR decomposition by deploying the Gram-Schmidt process described above.

In [1]:
import numpy as np
from scipy.linalg import qr


def QR_Decomposition(A):
    n, m = A.shape # get the shape of A

    Q = np.empty((n, n)) # initialize matrix Q
    u = np.empty((n, n)) # initialize matrix u

    u[:, 0] = A[:, 0]
    Q[:, 0] = u[:, 0] / np.linalg.norm(u[:, 0])

    for i in range(1, n):

        u[:, i] = A[:, i]
        for j in range(i):
            u[:, i] -= (A[:, i] @ Q[:, j]) * Q[:, j] # get each u vector

        Q[:, i] = u[:, i] / np.linalg.norm(u[:, i]) # compute each e vetor

    R = np.zeros((n, m))
    for i in range(n):
        for j in range(i, m):
            R[i, j] = A[:, j] @ Q[:, i]

    return Q, R

The preceding code is fine but can benefit from some further housekeeping. We want to do this because later in this notebook we want to compare results from using our homemade code above with the code for a QR that the Python scipy package delivers.

There can be be sign differences between the $Q$ and $R$ matrices produced by different numerical algorithms. All of these are valid QR decompositions because of how the sign differences cancel out when we compute $QR$.

However, to make the results from our homemade function and the QR module in scipy comparable, let’s require that $Q$ have positive diagonal entries. We do this by adjusting the signs of the columns in $Q$ and the rows in $R$ appropriately.

To accomplish this we’ll define a pair of functions.

In [2]:
def diag_sign(A):
    "Compute the signs of the diagonal of matrix A"

    D = np.diag(np.sign(np.diag(A)))

    return D

def adjust_sign(Q, R):
    """
    Adjust the signs of the columns in Q and rows in R to
    impose positive diagonal of Q
    """

    D = diag_sign(Q)

    Q[:, :] = Q @ D
    R[:, :] = D @ R

    return Q, R

In [3]:
# Example
A = np.array([[1.0, 1.0, 0.0], [1.0, 0.0, 1.0], [0.0, 1.0, 1.0]])
# A = np.array([[1.0, 0.5, 0.2], [0.5, 0.5, 1.0], [0.0, 1.0, 1.0]])
# A = np.array([[1.0, 0.5, 0.2], [0.5, 0.5, 1.0]])

A

array([[1., 1., 0.],
       [1., 0., 1.],
       [0., 1., 1.]])

In [4]:
Q, R = adjust_sign(*QR_Decomposition(A))

In [5]:
Q

array([[ 0.70710678, -0.40824829, -0.57735027],
       [ 0.70710678,  0.40824829,  0.57735027],
       [ 0.        , -0.81649658,  0.57735027]])

In [6]:
R

array([[ 1.41421356,  0.70710678,  0.70710678],
       [ 0.        , -1.22474487, -0.40824829],
       [ 0.        ,  0.        ,  1.15470054]])

In [7]:
Q_scipy, R_scipy = adjust_sign(*qr(A))

In [8]:
print('Our Q: \n', Q)
print('\n')
print('Scipy Q: \n', Q_scipy)

Our Q: 
 [[ 0.70710678 -0.40824829 -0.57735027]
 [ 0.70710678  0.40824829  0.57735027]
 [ 0.         -0.81649658  0.57735027]]


Scipy Q: 
 [[ 0.70710678 -0.40824829 -0.57735027]
 [ 0.70710678  0.40824829  0.57735027]
 [ 0.         -0.81649658  0.57735027]]


In [9]:
print('Our R: \n', R)
print('\n')
print('Scipy R: \n', R_scipy)

Our R: 
 [[ 1.41421356  0.70710678  0.70710678]
 [ 0.         -1.22474487 -0.40824829]
 [ 0.          0.          1.15470054]]


Scipy R: 
 [[ 1.41421356  0.70710678  0.70710678]
 [ 0.         -1.22474487 -0.40824829]
 [ 0.          0.          1.15470054]]


The above outcomes give us the good news that our homemade function agrees with what scipy produces. Now let’s do a QR decomposition for a rectangular matrix $A$ that is $n\times$ with $m >n$.

In [10]:
A = np.array([[1, 3, 4], [2, 0, 9]])

In [11]:
Q, R = adjust_sign(*QR_Decomposition(A))
Q, R

(array([[ 0.4472136 , -0.89442719],
        [ 0.89442719,  0.4472136 ]]),
 array([[ 2.23606798,  1.34164079,  9.8386991 ],
        [ 0.        , -2.68328157,  0.4472136 ]]))

In [12]:
Q_scipy, R_scipy = adjust_sign(*qr(A))
Q_scipy, R_scipy

(array([[ 0.4472136 , -0.89442719],
        [ 0.89442719,  0.4472136 ]]),
 array([[ 2.23606798,  1.34164079,  9.8386991 ],
        [ 0.        , -2.68328157,  0.4472136 ]]))

## Using QR Decomposition to Compute Eigenvalues

Now for a useful fact about the QR algorithm. The following iterations on the QR decomposition can be used to compute **eigenvalues** of a **square** matrix $A$.

Here is the algorithm:

1. Set $A_0=A$ and form $A_0 = Q_0R_0$
1. Form $A_1 = R_0Q_0$. Note that $A_1$ is similar to $A_0$ (easy to verify) and so has the same eigenvalues.
1. Form $A_1 = Q_1R_1$ (i.e., form the $QR$ decomposition of $A_1$).
1. Form $A_2 = R_1Q_1$ and then $A_2 = Q_2R_2$.
1. Iterate to convergence.
1. Compute eigenvalues of $A$ and compare them to the diagonal values of the limiting $A_n$ found from this process.

**Remark**: this algorithm is close to one of the most efficient ways of computing eigenvalues!

Let’s write some Python code to try out the algorithm

In [13]:
def QR_eigvals(A, tol=1e-12, maxiter=1000):
    "Find the eigenvalues of A using QR decomposition."

    A_old = np.copy(A)
    A_new = np.copy(A)

    diff = np.inf
    i = 0
    while (diff > tol) and (i < maxiter):
        A_old[:, :] = A_new
        Q, R = QR_Decomposition(A_old)

        A_new[:, :] = R @ Q

        diff = np.abs(A_new - A_old).max()
        i += 1

    eigvals = np.diag(A_new)

    return eigvals

Now let’s try the code and compare the results with what `scipy.linalg.eigvals` gives us

Here goes

In [14]:
# experiment this with one random A matrix
A = np.random.random((3, 3))

In [15]:
sorted(QR_eigvals(A))

[-0.11598607417977123, 0.2498725453213848, 1.3376804202664323]

Compare with the scipy package.

In [16]:
sorted(np.linalg.eigvals(A))

[-0.11598607417977602, 0.2498725453213847, 1.337680420266434]

## $QR$ and PCA

There are interesting connections between the $QR$ decomposition and principal components analysis (PCA). Here are some. 
1. Let $X'$ be a $k\times n$ random matrix where the $j$ th column is a random draw from 
$\mathcal N(\mu, \Sigma)$ where $\mu$ is $k\times 1$ vector of means and $\Sigma$ is a $k\times k$ covariance matrix. We want $n >> k$ - this is an “econometrics example”.
1. Form $X' = QR$ where $Q$ is $k\times k$ an $R$ is $k\times n$ and $R$ is $k\times n$.
1. Form the eigenvalues of $RR'$, i.e., we'll compute $RR' = \tilde P\Lambda \tilde P'$.
1. Form $XX' = Q\tilde P\Lambda \tilde P'Q'$ and compare it with the eigen decomposition $XX' = P\hat\Lambda P'$.
1. It will turn out that that $\Lambda = \hat\Lambda$ and that $P = Q\tilde P$.

Let’s verify conjecture 5 with some Python code. Start by simulating a random $(n,k)$ matrix $X$.

In [17]:
k = 5
n = 1000

# generate some random moments
mu = np.random.random(size=k)
C = np.random.random((k, k))
Sigma = C.T @ C

In [20]:
# X is random matrix where each column follows multivariate normal dist.
X = np.random.multivariate_normal(mu, Sigma, size=n)

In [21]:
X.shape

(1000, 5)

Let’s apply the QR decomposition to $X'$

In [22]:
Q, R = adjust_sign(*QR_Decomposition(X.T))

Check the shapes of $Q$ and $R$

In [23]:
Q.shape, R.shape

((5, 5), (5, 1000))

Now we can construct $RR' = \tilde P\Lambda \tilde P'$ and form an eigen decomposition.

In [25]:
RR = R @ R.T

lambda_, P_tilde = np.linalg.eigh(RR)
Λ = np.diag(lambda_)

We can also apply the decomposition to $XX' = P\hat\Lambda P'$.

In [26]:
XX = X.T @ X

lambda__hat, P = np.linalg.eigh(XX)
Λ_hat = np.diag(lambda__hat)

Compare the eigenvalues that are on the diagonals of $\Lambda$ and $\hat\Lambda$

In [27]:
lambda_, lambda__hat

(array([  19.53147905,  273.12808368,  624.22626792, 1187.93030619,
        5813.78081955]),
 array([  19.53147905,  273.12808368,  624.22626792, 1187.93030619,
        5813.78081955]))

Let’s compare $P$ and $Q\tilde P$. Again we need to be careful about sign differences between the columns of $P$ and $Q\tilde P$.

In [28]:
QP_tilde = Q @ P_tilde

np.abs(P @ diag_sign(P) - QP_tilde @ diag_sign(QP_tilde)).max()

1.2212453270876722e-15

Let’s verify that $X'X$ can be decomposed as $Q\tilde P\Lambda \tilde P'Q'$.

In [29]:
QPΛPQ = Q @ P_tilde @ Λ @ P_tilde.T @ Q.T

In [30]:
np.abs(QPΛPQ - XX).max()

2.5011104298755527e-12